In [46]:
import warnings
warnings.filterwarnings("ignore")


In [47]:
import numpy as np
import pandas as pd
import pickle

def preprocess_new_record(new_record, pipeline_path='Data/preprocessing_pipeline.pkl'):
    """
    Apply the same preprocessing to new records
    
    Parameters:
    -----------
    new_record : dict or pd.DataFrame
        New record(s) to preprocess
    pipeline_path : str
        Path to saved preprocessing pipeline
        
    Returns:
    --------
    pd.DataFrame : Preprocessed record matching training data format
    """
    
    # Load pipeline
    with open(pipeline_path, 'rb') as f:
        pipeline = pickle.load(f)
    
    # Convert to DataFrame if needed
    if isinstance(new_record, dict):
        df_new = pd.DataFrame([new_record])
    else:
        df_new = new_record.copy()
    
    # Step 1: Normalize Unknown-like responses
    df_new = df_new.replace({
        "Don't know": "Unknown", "Refused": "Unknown", 
        "Not Applicable": "Unknown", "N/A": "Unknown", 
        "Unknown/NA": "Unknown"
    })
    
    # Step 2: Binary encoding
    for col in pipeline['binary_cols']:
        if col not in df_new.columns:
            continue
        
        if col == "Has_diabetes":
            mapping = pipeline['binary_mappings']["Has_diabetes"]
        elif col == "Received_Hepatitis_A_Vaccine":
            mapping = pipeline['binary_mappings']["Received_Hepatitis_A_Vaccine"]
        else:
            mapping = pipeline['binary_mappings']["default"]
        
        df_new[col] = df_new[col].map(mapping)
    
    # Step 3: Ordinal encoding
    for col, encoder in pipeline['ordinal_encoders'].items():
        if col in df_new.columns:
            df_new[col] = encoder.transform(df_new[[col]])
    
    # Step 4: One-hot encoding
    for base_col in pipeline['ohe_cols']:
        if base_col in df_new.columns:
            # Get dummies for this column
            dummies = pd.get_dummies(df_new[base_col], prefix=base_col, dtype=int)
            
            # Add any missing columns from training
            for train_col in pipeline['ohe_column_names']:
                if base_col in train_col and train_col not in dummies.columns:
                    dummies[train_col] = 0
            
            # Remove extra columns not in training
            cols_to_keep = [col for col in dummies.columns 
                           if col in pipeline['ohe_column_names']]
            dummies = dummies[cols_to_keep]
            
            # Add to dataframe
            df_new = pd.concat([df_new.drop(columns=[base_col]), dummies], axis=1)
    
    # Step 5: Apply log transformation to skewed columns
    for col in pipeline['skewed_cols']:
        if col in df_new.columns:
            df_new[col] = np.log1p(df_new[col].clip(lower=0))
    
    # Step 6: Ensure all columns from training exist
    for col in pipeline['all_columns']:
        if col not in df_new.columns:
            df_new[col] = 0  # Add missing columns with default value
    
    # Step 7: Reorder columns to match training data
    df_new = df_new[pipeline['all_columns']]
    
    # Step 8: Apply scaling
    df_new[pipeline['cols_to_scale']] = pipeline['scaler'].transform(
        df_new[pipeline['cols_to_scale']]
    )
    
    return df_new

In [48]:
# Example new patient record
test_df = pd.read_csv('Data/test_dataset.csv')
df_scaled = pd.read_csv('Data/df_scaled.csv')
df_scaled.drop(columns='Unnamed: 0', inplace=True)

# Preprocess the new record
processed_patient = preprocess_new_record(test_df)

print(f"Shape of processed record: {processed_patient.shape}")
print(f"Matches training data shape: {processed_patient.shape[1] == df_scaled.shape[1]}")

Shape of processed record: (1050, 55)
Matches training data shape: True


# Dim Reduction UMAP

In [49]:
# !pip install umap-learn


In [50]:
import joblib
umap = joblib.load('Data/umap_model.pkl')
umap_test = umap.transform(processed_patient)
umap_test_df = pd.DataFrame(umap_test, columns=[f'UMAP{i+1}' for i in range(umap_test.shape[1])])
umap_test_df.head()

print(umap_test_df.shape)
print(umap_test_df.head())

Sat Nov 22 16:16:14 2025 Building and compiling search function


Epochs completed:   0%|            0/100 [00:00]

	completed  0  /  100 epochs
	completed  10  /  100 epochs
	completed  20  /  100 epochs
	completed  30  /  100 epochs
	completed  40  /  100 epochs
	completed  50  /  100 epochs
	completed  60  /  100 epochs
	completed  70  /  100 epochs
	completed  80  /  100 epochs
	completed  90  /  100 epochs
(1050, 20)
       UMAP1      UMAP2     UMAP3     UMAP4     UMAP5     UMAP6     UMAP7  \
0  13.024805  13.073879  5.157461  2.991746  5.764147  5.317441  6.363474   
1  -4.150143   5.798073  5.107178  6.727241  4.209681  2.940264 -1.061531   
2  12.650412  12.129565  5.002039  3.329209  5.700506  6.181603  5.701279   
3  12.410887   9.398935  4.929136  3.380293  5.901017  5.174147  5.899176   
4  -4.033573   5.764928  5.089195  6.560227  4.229160  2.989349 -0.971349   

      UMAP8     UMAP9    UMAP10    UMAP11    UMAP12    UMAP13    UMAP14  \
0  6.094333  1.970684  3.853272  6.677246  4.087493  3.857002  2.361790   
1  5.780765  7.538804  4.759531  6.841758  3.087821  8.424713  8.717398   
2 

# KMeans

In [51]:
import joblib
import pandas as pd
import numpy as np

# ============================================================
# LOAD CLUSTERING MODELS
# ============================================================
print("="*70)
print("LOADING CLUSTERING MODELS")
print("="*70)

kmeans_final = joblib.load('Models/kmeans_umap_initial4_model.pkl')
kmeans_c0    = joblib.load('Models/kmeans_umap_c0.pkl')  
kmeans_c3    = joblib.load('Models/kmeans_umap_c3.pkl')

k_split_c0 = 3
k_split_c3 = 2

print("✓ All models loaded successfully")

# ============================================================
# ASSIGN CLUSTERS TO ALL TEST PATIENTS
# ============================================================
print("\n" + "="*70)
print("ASSIGNING CLUSTERS TO TEST PATIENTS")
print("="*70)

print(f"Test data shape: {umap_test_df.shape}")

# Make sure columns match what KMeans was trained on (optional but safer)
if hasattr(kmeans_final, "feature_names_in_"):
    expected_cols = list(kmeans_final.feature_names_in_)
    umap_test_df = umap_test_df[expected_cols]

# <<< NEW: cast to the same dtype as the model's cluster centers
dtype = kmeans_final.cluster_centers_.dtype
X_test = umap_test_df.to_numpy().astype(dtype)

# Step 1: Get initial cluster assignments (4 clusters)
initial_clusters = kmeans_final.predict(X_test)

# Step 2: Apply hierarchical logic to get FINAL clusters (0..6)
final_clusters = []
subcluster_assignments = []

for idx in range(X_test.shape[0]):
    main_cluster = initial_clusters[idx]
    
    # use the numpy slice so dtype stays consistent
    patient_features = X_test[idx:idx+1, :]   # shape (1, n_features)
    
    if main_cluster == 0:
        # Initial Cluster 0 splits into subclusters → Final clusters 0, 1, 2
        subcluster_id = kmeans_c0.predict(patient_features)[0]  # {0,1,2}
        final_cluster = 0 + subcluster_id                      # 0,1,2
        final_clusters.append(int(final_cluster))
        subcluster_assignments.append(int(subcluster_id))
        
    elif main_cluster == 1:
        # Initial Cluster 1 stays as final cluster 3
        final_clusters.append(3)
        subcluster_assignments.append(None)
        
    elif main_cluster == 2:
        # Initial Cluster 2 stays as final cluster 4
        final_clusters.append(4)
        subcluster_assignments.append(None)
        
    elif main_cluster == 3:
        # Initial Cluster 3 splits into subclusters → Final clusters 5, 6
        subcluster_id = kmeans_c3.predict(patient_features)[0]  # {0,1}
        final_cluster = 5 + subcluster_id                      # 5,6
        final_clusters.append(int(final_cluster))
        subcluster_assignments.append(int(subcluster_id))

# Attach results back to a DataFrame (we can reuse umap_test_df)
umap_test_df_assigned = umap_test_df.copy()
umap_test_df_assigned['Initial_Cluster_UMAP'] = initial_clusters
umap_test_df_assigned['Subcluster_UMAP']      = subcluster_assignments
umap_test_df_assigned['Final_Cluster_UMAP']   = final_clusters

# ============================================================
# DISPLAY RESULTS
# ============================================================
print("\n" + "="*70)
print("UMAP CLUSTER ASSIGNMENT SUMMARY")
print("="*70)

print("\nInitial cluster distribution (4 clusters):")
print(umap_test_df_assigned['Initial_Cluster_UMAP'].value_counts().sort_index())

print("\nFinal cluster distribution (7 clusters):")
print(umap_test_df_assigned['Final_Cluster_UMAP'].value_counts().sort_index())

print("\nCluster mapping (UMAP-based):")
print("  Final Cluster 0: Initial Cluster 0 → Subcluster 0")
print("  Final Cluster 1: Initial Cluster 0 → Subcluster 1")
print("  Final Cluster 2: Initial Cluster 0 → Subcluster 2")
print("  Final Cluster 3: Initial Cluster 1 (no subclustering)")
print("  Final Cluster 4: Initial Cluster 2 (no subclustering)")
print("  Final Cluster 5: Initial Cluster 3 → Subcluster 0")
print("  Final Cluster 6: Initial Cluster 3 → Subcluster 1")

print("\nFirst 20 patient assignments:")
print(umap_test_df_assigned[['Initial_Cluster_UMAP', 'Subcluster_UMAP', 'Final_Cluster_UMAP']].head(20))

# Save results
umap_test_df_assigned.to_csv('Data/test_patients_with_clusters_kmeans_umap.csv', index=False)
print("\n✓ Results saved to 'Data/test_patients_with_clusters_kmeans_umap.csv'")


LOADING CLUSTERING MODELS
✓ All models loaded successfully

ASSIGNING CLUSTERS TO TEST PATIENTS
Test data shape: (1050, 20)

UMAP CLUSTER ASSIGNMENT SUMMARY

Initial cluster distribution (4 clusters):
Initial_Cluster_UMAP
0    574
1    173
2    132
3    171
Name: count, dtype: int64

Final cluster distribution (7 clusters):
Final_Cluster_UMAP
0    457
1      2
2    115
3    173
4    132
5     46
6    125
Name: count, dtype: int64

Cluster mapping (UMAP-based):
  Final Cluster 0: Initial Cluster 0 → Subcluster 0
  Final Cluster 1: Initial Cluster 0 → Subcluster 1
  Final Cluster 2: Initial Cluster 0 → Subcluster 2
  Final Cluster 3: Initial Cluster 1 (no subclustering)
  Final Cluster 4: Initial Cluster 2 (no subclustering)
  Final Cluster 5: Initial Cluster 3 → Subcluster 0
  Final Cluster 6: Initial Cluster 3 → Subcluster 1

First 20 patient assignments:
    Initial_Cluster_UMAP  Subcluster_UMAP  Final_Cluster_UMAP
0                      0              0.0                   0
1       

# Cluster Validation

In [52]:
import numpy as np
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

print("="*70)
print("CLUSTER ASSIGNMENT VALIDATION")
print("="*70)

CLUSTER ASSIGNMENT VALIDATION


In [53]:
# Load the training PCA data
df_umap = pd.read_csv('Data/umap_dataset.csv')

print(f"Training data loaded: {df_umap.shape}")
print(f"Columns: {df_umap.columns.tolist()}")

Training data loaded: (9442, 20)
Columns: ['UMAP1', 'UMAP2', 'UMAP3', 'UMAP4', 'UMAP5', 'UMAP6', 'UMAP7', 'UMAP8', 'UMAP9', 'UMAP10', 'UMAP11', 'UMAP12', 'UMAP13', 'UMAP14', 'UMAP15', 'UMAP16', 'UMAP17', 'UMAP18', 'UMAP19', 'UMAP20']


In [54]:
print("\n[Metric 1] Within-Cluster Variance Stability")
print("-"*70)

def calculate_within_cluster_variance(data, labels, centroids):
    """Calculate within-cluster variance for each cluster"""
    variances = {}
    for cluster_id in np.unique(labels):
        cluster_points = data[labels == cluster_id]
        centroid = centroids[cluster_id]
        variance = np.mean(np.sum((cluster_points - centroid)**2, axis=1))
        variances[cluster_id] = variance
    return variances

# For initial 4 clusters - calculate on TRAINING data
train_initial_labels = kmeans_final.labels_  # Training labels
train_centroids = kmeans_final.cluster_centers_

# Calculate training variance
train_variances = calculate_within_cluster_variance(
    df_umap.drop(columns=['Cluster', 'Cluster_Original', 'Cluster_Refined'], errors='ignore').values,
    train_initial_labels,
    train_centroids
)

# Calculate test variance
test_initial_labels = initial_clusters
# ✅ Use correct object (umap_test_df if DataFrame, else pca_test if array)
test_data = umap_test_df.values if 'umap_test_df' in locals() else umap_test

test_variances = calculate_within_cluster_variance(
    test_data,
    test_initial_labels,
    train_centroids
)

print("\nWithin-cluster variance comparison (Initial 4 clusters):")
print(f"{'Cluster':<10} {'Train Variance':<20} {'Test Variance':<20} {'% Change':<15}")
print("-"*70)

for cluster_id in sorted(train_variances.keys()):
    train_var = train_variances[cluster_id]
    test_var = test_variances.get(cluster_id, np.nan)
    
    if not np.isnan(test_var):
        pct_change = ((test_var - train_var) / train_var) * 100
        status = "✓ Stable" if abs(pct_change) < 20 else "⚠ Check"
        print(f"{cluster_id:<10} {train_var:<20.4f} {test_var:<20.4f} {pct_change:>+.2f}%  {status}")
    else:
        print(f"{cluster_id:<10} {train_var:<20.4f} {'No test samples':<20} {'N/A':<15}")



[Metric 1] Within-Cluster Variance Stability
----------------------------------------------------------------------

Within-cluster variance comparison (Initial 4 clusters):
Cluster    Train Variance       Test Variance        % Change       
----------------------------------------------------------------------
0          46.2789              46.8613              +1.26%  ✓ Stable
1          59.9360              61.5086              +2.62%  ✓ Stable
2          31.4401              30.6412              -2.54%  ✓ Stable
3          43.4805              37.4921              -13.77%  ✓ Stable


In [55]:
import numpy as np

def compute_centroids_and_variances(X, labels):
    """Return centroids and within-cluster variances for given labels."""
    centroids = {}
    variances = {}
    for cid in np.unique(labels):
        mask = (labels == cid)
        pts = X[mask]
        if pts.size == 0:
            continue
        c = pts.mean(axis=0)
        centroids[cid] = c
        d2 = np.sum((pts - c)**2, axis=1)
        variances[cid] = d2.mean()
    return centroids, variances

# ============================================================
# BUILD TRAIN UMAP MATRIX + FINAL CLUSTER LABELS
# ============================================================
umap_cols = [c for c in df_umap.columns if c.startswith("UMAP")]
train_umap_df = df_umap[umap_cols]

# Ensure same column order as used to train kmeans_final
if hasattr(kmeans_final, "feature_names_in_"):
    expected_cols = list(kmeans_final.feature_names_in_)
    train_umap_df = train_umap_df[expected_cols]

# Match dtype to model
X_train = train_umap_df.to_numpy().astype(kmeans_final.cluster_centers_.dtype)
train_initial = kmeans_final.labels_  # 0..3 on train

train_final = np.empty_like(train_initial)

for i, main in enumerate(train_initial):
    point = X_train[i:i+1, :]  # shape (1, d)
    
    if main == 0:
        # initial 0 → subclusters 0,1,2 → final 0,1,2
        sub = kmeans_c0.predict(point)[0]      # 0,1,2
        final = 0 + sub
    elif main == 1:
        # initial 1 → final 3
        final = 3
    elif main == 2:
        # initial 2 → final 4
        final = 4
    elif main == 3:
        # initial 3 → subclusters 0,1 → final 5,6
        sub = kmeans_c3.predict(point)[0]      # 0,1
        final = 5 + sub
    
    train_final[i] = int(final)

# ============================================================
# TRAIN VARIANCES (FINAL CLUSTERS)
# ============================================================
cent_train, var_train = compute_centroids_and_variances(X_train, train_final)

# ============================================================
# TEST VARIANCES (FINAL CLUSTERS)
# ============================================================
test_umap_df = umap_test_df_assigned[umap_cols]
if hasattr(kmeans_final, "feature_names_in_"):
    test_umap_df = test_umap_df[expected_cols]

X_test = test_umap_df.to_numpy().astype(kmeans_final.cluster_centers_.dtype)
test_final = umap_test_df_assigned["Final_Cluster_UMAP"].to_numpy()

_, var_test = compute_centroids_and_variances(X_test, test_final)

# ============================================================
# COMPARISON: TRAIN vs TEST WITH % CHANGE
# ============================================================
print("\nWithin-cluster variance comparison (Final UMAP clusters):")
print(f"{'Cluster':<8} {'Train Var':<18} {'Test Var':<18} {'% Change':<12}")
print("-"*60)

for cid in sorted(var_train.keys()):
    tv = var_train[cid]
    sv = var_test.get(cid, np.nan)
    
    if np.isnan(sv):
        print(f"{cid:<8} {tv:<18.4f} {'No test pts':<18} {'N/A':<12}")
    else:
        pct = ((sv - tv) / tv * 100) if tv != 0 else np.nan
        tag = "✓ Stable" if (not np.isnan(pct) and abs(pct) < 25) else "⚠ Check"
        print(f"{cid:<8} {tv:<18.4f} {sv:<18.4f} {pct:>+8.2f}%  {tag}")



Within-cluster variance comparison (Final UMAP clusters):
Cluster  Train Var          Test Var           % Change    
------------------------------------------------------------
0        1.4662             1.8775               +28.05%  ⚠ Check
1        8.3668             0.0002              -100.00%  ⚠ Check
2        1.0266             0.9766                -4.88%  ✓ Stable
3        1.6836             2.0313               +20.65%  ✓ Stable
4        0.9731             0.7724               -20.63%  ✓ Stable
5        0.9464             0.4574               -51.66%  ⚠ Check
6        0.7392             0.7756                +4.92%  ✓ Stable


# Testing

In [56]:
print("\n[Metric 2] Cluster Separation & Decision Boundary Confidence")
print("-"*70)

def calculate_separation_metrics(X, labels, centroids):
    """
    For each point, calculate:
    1. Distance to assigned cluster centroid (d1)
    2. Distance to nearest other cluster centroid (d2)
    3. Separation ratio: (d2 - d1) / d1 (higher = more confident assignment)
    """
    results = {
        'cluster': [],
        'assigned_distance': [],
        'nearest_other_distance': [],
        'separation_ratio': [],
        'nearest_other_cluster': []
    }
    
    for i, point in enumerate(X):
        assigned_cluster = labels[i]
        
        # Distance to assigned centroid
        d1 = np.linalg.norm(point - centroids[assigned_cluster])
        
        # Find nearest other cluster
        other_distances = {}
        for cid, centroid in centroids.items():
            if cid != assigned_cluster:
                other_distances[cid] = np.linalg.norm(point - centroid)
        
        nearest_other_cluster = min(other_distances, key=other_distances.get)
        d2 = other_distances[nearest_other_cluster]
        
        # Separation ratio: higher means more confident assignment
        sep_ratio = (d2 - d1) / d1 if d1 > 0 else np.inf
        
        results['cluster'].append(assigned_cluster)
        results['assigned_distance'].append(d1)
        results['nearest_other_distance'].append(d2)
        results['separation_ratio'].append(sep_ratio)
        results['nearest_other_cluster'].append(nearest_other_cluster)
    
    return pd.DataFrame(results)

# Calculate for train
sep_train = calculate_separation_metrics(X_train, train_final, cent_train)

# Calculate for test
sep_test = calculate_separation_metrics(X_test, test_final, cent_train)

# Summary by cluster
print("\nTrain Set - Cluster Assignment Confidence:")
print(f"{'Cluster':<8} {'Avg Sep Ratio':<18} {'% Ambiguous (<0.3)':<20} {'Confused With':<15}")
print("-"*70)

for cid in sorted(sep_train['cluster'].unique()):
    cluster_data = sep_train[sep_train['cluster'] == cid]
    avg_sep = cluster_data['separation_ratio'].mean()
    pct_ambiguous = (cluster_data['separation_ratio'] < 0.3).mean() * 100
    
    # Most common confused cluster
    confused_with = cluster_data['nearest_other_cluster'].mode()[0] if len(cluster_data) > 0 else 'N/A'
    
    status = "✓ Clear" if pct_ambiguous < 10 else ("⚠ Moderate" if pct_ambiguous < 25 else "❌ Fuzzy")
    print(f"{cid:<8} {avg_sep:<18.3f} {pct_ambiguous:>6.1f}%  {status:<12} Cluster {confused_with}")

print("\nTest Set - Cluster Assignment Confidence:")
print(f"{'Cluster':<8} {'Avg Sep Ratio':<18} {'% Ambiguous (<0.3)':<20} {'Confused With':<15}")
print("-"*70)

for cid in sorted(sep_test['cluster'].unique()):
    cluster_data = sep_test[sep_test['cluster'] == cid]
    avg_sep = cluster_data['separation_ratio'].mean()
    pct_ambiguous = (cluster_data['separation_ratio'] < 0.3).mean() * 100
    
    confused_with = cluster_data['nearest_other_cluster'].mode()[0] if len(cluster_data) > 0 else 'N/A'
    
    status = "✓ Clear" if pct_ambiguous < 10 else ("⚠ Moderate" if pct_ambiguous < 25 else "❌ Fuzzy")
    print(f"{cid:<8} {avg_sep:<18.3f} {pct_ambiguous:>6.1f}%  {status:<12} Cluster {confused_with}")

# Identify most ambiguous assignments in test set
print("\n⚠ Most Ambiguous Assignments in Test Set (Separation Ratio < 0.2):")
ambiguous = sep_test[sep_test['separation_ratio'] < 0.2].copy()
ambiguous = ambiguous.sort_values('separation_ratio')
print(f"Found {len(ambiguous)} ambiguous assignments ({len(ambiguous)/len(sep_test)*100:.1f}% of test set)")
print(f"\n{'Index':<8} {'Assigned':<10} {'Could be':<10} {'Sep Ratio':<12}")
print("-"*50)
for idx, row in ambiguous.head(10).iterrows():
    print(f"{idx:<8} {int(row['cluster']):<10} {int(row['nearest_other_cluster']):<10} {row['separation_ratio']:<12.3f}")


[Metric 2] Cluster Separation & Decision Boundary Confidence
----------------------------------------------------------------------

Train Set - Cluster Assignment Confidence:
Cluster  Avg Sep Ratio      % Ambiguous (<0.3)   Confused With  
----------------------------------------------------------------------
0        1.035                28.9%  ❌ Fuzzy      Cluster 5
1        0.393                32.3%  ❌ Fuzzy      Cluster 2
2        3.588                18.8%  ⚠ Moderate   Cluster 0
3        32.003                0.4%  ✓ Clear      Cluster 1
4        4.124                 2.3%  ✓ Clear      Cluster 5
5        1.356                 9.0%  ✓ Clear      Cluster 0
6        2.191                 0.1%  ✓ Clear      Cluster 5

Test Set - Cluster Assignment Confidence:
Cluster  Avg Sep Ratio      % Ambiguous (<0.3)   Confused With  
----------------------------------------------------------------------
0        0.841                36.8%  ❌ Fuzzy      Cluster 5
1        0.662              

In [57]:
print("\n[Metric 3] Feature Contribution to Cluster Assignment")
print("-"*70)

def calculate_feature_importance_for_assignment(X, labels, centroids, feature_names):
    """
    For each cluster, calculate which features most distinguish it from others.
    Uses the ratio of within-cluster variance to between-cluster variance per feature.
    """
    feature_importance = {}
    
    for cid in np.unique(labels):
        cluster_points = X[labels == cid]
        other_points = X[labels != cid]
        
        if len(cluster_points) == 0:
            continue
        
        # For each feature, calculate separation
        importance_scores = []
        for f in range(X.shape[1]):
            cluster_vals = cluster_points[:, f]
            other_vals = other_points[:, f]
            
            # Effect size (Cohen's d)
            mean_diff = abs(cluster_vals.mean() - other_vals.mean())
            pooled_std = np.sqrt((cluster_vals.std()**2 + other_vals.std()**2) / 2)
            
            if pooled_std > 0:
                cohens_d = mean_diff / pooled_std
            else:
                cohens_d = 0
            
            importance_scores.append(cohens_d)
        
        feature_importance[cid] = pd.DataFrame({
            'Feature': feature_names,
            'Importance': importance_scores
        }).sort_values('Importance', ascending=False)
    
    return feature_importance

# Get feature names (original features before UMAP)
# You'll need to use your original feature columns here
# For now, if you want to do this on UMAP dimensions:
umap_feature_names = [f"UMAP_{i}" for i in range(X_train.shape[1])]

feat_importance_train = calculate_feature_importance_for_assignment(
    X_train, train_final, cent_train, umap_feature_names
)

print("\nTop 3 Distinguishing Features per Cluster (Train Set):")
print("-"*70)
for cid in sorted(feat_importance_train.keys()):
    print(f"\nCluster {cid}:")
    top_features = feat_importance_train[cid].head(3)
    for _, row in top_features.iterrows():
        print(f"  {row['Feature']:<15} Importance: {row['Importance']:.3f}")

# For TEST set - check if feature importance pattern holds
feat_importance_test = calculate_feature_importance_for_assignment(
    X_test, test_final, cent_train, umap_feature_names
)

print("\n\nFeature Importance Consistency (Train vs Test):")
print(f"{'Cluster':<8} {'Train Top Feat':<20} {'Test Top Feat':<20} {'Match?':<10}")
print("-"*70)
for cid in sorted(feat_importance_train.keys()):
    if cid in feat_importance_test:
        train_top = feat_importance_train[cid].iloc[0]['Feature']
        test_top = feat_importance_test[cid].iloc[0]['Feature']
        match = "✓" if train_top == test_top else "✗"
        print(f"{cid:<8} {train_top:<20} {test_top:<20} {match:<10}")


[Metric 3] Feature Contribution to Cluster Assignment
----------------------------------------------------------------------

Top 3 Distinguishing Features per Cluster (Train Set):
----------------------------------------------------------------------

Cluster 0:
  UMAP_8          Importance: 1.524
  UMAP_1          Importance: 1.430
  UMAP_15         Importance: 1.391

Cluster 1:
  UMAP_11         Importance: 2.421
  UMAP_10         Importance: 2.117
  UMAP_9          Importance: 1.947

Cluster 2:
  UMAP_15         Importance: 1.672
  UMAP_10         Importance: 1.653
  UMAP_1          Importance: 1.254

Cluster 3:
  UMAP_0          Importance: 53.883
  UMAP_12         Importance: 16.411
  UMAP_3          Importance: 14.678

Cluster 4:
  UMAP_17         Importance: 2.511
  UMAP_11         Importance: 2.149
  UMAP_10         Importance: 1.012

Cluster 5:
  UMAP_7          Importance: 2.082
  UMAP_18         Importance: 1.280
  UMAP_9          Importance: 1.180

Cluster 6:
  UMAP_7    

In [58]:
print("\n[Metric 4] Cluster Membership Stability")
print("-"*70)

from sklearn.utils import resample

def calculate_stability_score(X, labels, centroids, n_bootstrap=50):
    """
    For each point, see how often it gets assigned to the same cluster
    when we add noise or bootstrap sample the data.
    """
    n_samples = X.shape[0]
    stability_scores = np.zeros(n_samples)
    
    for _ in range(n_bootstrap):
        # Add small Gaussian noise
        noise_std = 0.05 * X.std(axis=0)  # 5% noise
        X_noisy = X + np.random.normal(0, noise_std, X.shape)
        
        # Reassign to nearest centroid
        new_labels = np.array([
            min(centroids.keys(), 
                key=lambda c: np.linalg.norm(X_noisy[i] - centroids[c]))
            for i in range(n_samples)
        ])
        
        # Check if assignment matches original
        stability_scores += (new_labels == labels).astype(int)
    
    return stability_scores / n_bootstrap

# Calculate stability for test set
stability_test = calculate_stability_score(X_test, test_final, cent_train, n_bootstrap=50)

# Summary by cluster
stability_df = pd.DataFrame({
    'cluster': test_final,
    'stability': stability_test
})

print("\nCluster Assignment Stability (Test Set):")
print(f"{'Cluster':<8} {'Mean Stability':<18} {'% Unstable (<0.7)':<20} {'Status':<10}")
print("-"*70)

for cid in sorted(stability_df['cluster'].unique()):
    cluster_stab = stability_df[stability_df['cluster'] == cid]['stability']
    mean_stab = cluster_stab.mean()
    pct_unstable = (cluster_stab < 0.7).mean() * 100
    
    status = "✓ Robust" if pct_unstable < 10 else ("⚠ Moderate" if pct_unstable < 25 else "❌ Fragile")
    print(f"{cid:<8} {mean_stab:<18.3f} {pct_unstable:>6.1f}%  {status:<15} {status}")

print(f"\nOverall Test Set Stability: {stability_test.mean():.3f}")
print(f"Samples with stability < 0.7: {(stability_test < 0.7).sum()} ({(stability_test < 0.7).mean()*100:.1f}%)")


[Metric 4] Cluster Membership Stability
----------------------------------------------------------------------

Cluster Assignment Stability (Test Set):
Cluster  Mean Stability     % Unstable (<0.7)    Status    
----------------------------------------------------------------------
0        0.690                31.5%  ❌ Fragile       ❌ Fragile
1        1.000                 0.0%  ✓ Robust        ✓ Robust
2        0.026                97.4%  ❌ Fragile       ❌ Fragile
3        1.000                 0.0%  ✓ Robust        ✓ Robust
4        1.000                 0.0%  ✓ Robust        ✓ Robust
5        0.955                 6.5%  ✓ Robust        ✓ Robust
6        1.000                 0.0%  ✓ Robust        ✓ Robust

Overall Test Set Stability: 0.756
Samples with stability < 0.7: 259 (24.7%)


In [59]:
def explain_patient_cluster(
    patient_features,          # 1D numpy array of UMAP features
    centroids,                 # dict {cluster_id: centroid array}
    feature_names,             # list of feature names (UMAP1, UMAP2, ...)
    feature_importance_dict,   # dict {cluster_id: pd.DataFrame with Feature, Importance}
    stability_score=None       # optional stability score for this patient
):
    # 1️⃣ Compute distances to all centroids
    distances = {cid: np.linalg.norm(patient_features - c) for cid, c in centroids.items()}
    
    # Assigned cluster: nearest centroid
    assigned_cluster = min(distances, key=distances.get)
    distance_to_assigned = distances[assigned_cluster]
    
    # Nearest competing cluster
    other_distances = {cid: d for cid, d in distances.items() if cid != assigned_cluster}
    nearest_other_cluster = min(other_distances, key=other_distances.get)
    distance_to_other = other_distances[nearest_other_cluster]
    
    # Separation ratio
    separation_ratio = (distance_to_other - distance_to_assigned) / distance_to_assigned if distance_to_assigned > 0 else np.inf
    
    # 2️⃣ Top features pulling patient toward assigned cluster
    # Use feature_importance_dict from Metric 3 (cohen's d per feature)
    importance_df = feature_importance_dict.get(assigned_cluster, pd.DataFrame())
    if not importance_df.empty:
        # Match features with patient differences from centroid
        contributions = {}
        for _, row in importance_df.iterrows():
            feat = row['Feature']
            # Absolute difference from assigned cluster centroid
            feat_idx = feature_names.index(feat)
            contributions[feat] = abs(patient_features[feat_idx] - centroids[assigned_cluster][feat_idx]) * row['Importance']
        # Top 3 features
        top_features = sorted(contributions.items(), key=lambda x: x[1], reverse=True)[:3]
        top_features_list = [{'Feature': f, 'Contribution': round(c, 3)} for f, c in top_features]
    else:
        top_features_list = []

    # 3️⃣ Construct summary
    summary_text = (
        f"Patient is assigned to Cluster {assigned_cluster} because they are "
        f"{separation_ratio:.2f} separation ratio closer in UMAP space compared to Cluster {nearest_other_cluster}. "
    )
    if top_features_list:
        feat_names = ', '.join([f['Feature'] for f in top_features_list])
        summary_text += f"Key features driving this assignment include {feat_names}. "
    if stability_score is not None:
        summary_text += f"Cluster membership stability = {stability_score:.2f}."
    
    # 4️⃣ Return structured explanation
    return {
        'assigned_cluster': assigned_cluster,
        'nearest_other_cluster': nearest_other_cluster,
        'distance_to_assigned': distance_to_assigned,
        'distance_to_other': distance_to_other,
        'separation_ratio': separation_ratio,
        'top_features_pulling_to_assigned': top_features_list,
        'stability_score': stability_score,
        'summary': summary_text
    }

In [60]:
import pandas as pd

def load_umap_feature_map(csv_path):
    df = pd.read_csv(csv_path)
    umap_map = {}

    # For UMAP1 to UMAP20
    for i in range(1, 21):
        feature_col = f"UMAP{i}_Feature"
        corr_col = f"UMAP{i}_Correlation"

        if feature_col in df.columns and corr_col in df.columns:
            # Always take the FIRST row = highest-correlation feature
            best_feature = df.loc[0, feature_col]
            best_corr = df.loc[0, corr_col]

            # Store in normalized form "UMAP_i"
            umap_map[f"UMAP_{i}"] = {
                "feature": best_feature,
                "correlation": float(best_corr)
            }

    return umap_map


In [61]:
def translate_umap_to_original(result, umap_map):
    translated = []

    for item in result["top_features_pulling_to_assigned"]:
        umap_feature = item["Feature"]  # like "UMAP_8"
        contrib = item["Contribution"]

        if umap_feature in umap_map:
            translated.append({
                "Original_Feature": umap_map[umap_feature]["feature"],
                "Correlation": umap_map[umap_feature]["correlation"],
                "Contribution": contrib
            })
        else:
            print(f"[Warning] No mapped feature for {umap_feature}")

    # sort by contribution descending
    translated = sorted(translated, key=lambda x: x["Contribution"], reverse=True)
    return translated


In [62]:
def update_summary(result, original_features):
    if original_features:
        top_feats = ", ".join([f["Original_Feature"] for f in original_features[:3]])
    else:
        top_feats = "N/A"

    result["summary"] = (
        f"Patient is assigned to Cluster {result['assigned_cluster']} because they are "
        f"{result['separation_ratio']:.2f} separation ratio closer in UMAP space compared to Cluster "
        f"{result['nearest_other_cluster']}. Key original health features driving this assignment include "
        f"{top_feats}. Cluster membership stability = {result['stability_score']:.2f}."
    )

    return result


## Test Patient 0

In [63]:
# Example usage for first patient in test set
patient_idx = 0
patient_features = X_test[patient_idx]
stability = stability_test[patient_idx]
explanation = explain_patient_cluster(
    patient_features,
    cent_train,
    umap_feature_names,
    feat_importance_train,
    stability_score=stability
)

import pprint
pprint.pprint(explanation)

{'assigned_cluster': 0,
 'distance_to_assigned': 0.666987075804459,
 'distance_to_other': 1.8968030256899717,
 'nearest_other_cluster': 5,
 'separation_ratio': 1.8438377511321653,
 'stability_score': 1.0,
 'summary': 'Patient is assigned to Cluster 0 because they are 1.84 separation '
            'ratio closer in UMAP space compared to Cluster 5. Key features '
            'driving this assignment include UMAP_1, UMAP_8, UMAP_15. Cluster '
            'membership stability = 1.00.',
 'top_features_pulling_to_assigned': [{'Contribution': 0.321,
                                       'Feature': 'UMAP_1'},
                                      {'Contribution': 0.246,
                                       'Feature': 'UMAP_8'},
                                      {'Contribution': 0.245,
                                       'Feature': 'UMAP_15'}]}


In [64]:
umap_map = load_umap_feature_map("umap_health_features_with_correlations.csv")

orig_features = translate_umap_to_original(explanation, umap_map)

explanation["top_original_features"] = orig_features

explanation = update_summary(explanation, orig_features)

explanation


{'assigned_cluster': 0,
 'nearest_other_cluster': 5,
 'distance_to_assigned': 0.666987075804459,
 'distance_to_other': 1.8968030256899717,
 'separation_ratio': 1.8438377511321653,
 'top_features_pulling_to_assigned': [{'Feature': 'UMAP_1',
   'Contribution': 0.321},
  {'Feature': 'UMAP_8', 'Contribution': 0.246},
  {'Feature': 'UMAP_15', 'Contribution': 0.245}],
 'stability_score': 1.0,
 'summary': 'Patient is assigned to Cluster 0 because they are 1.84 separation ratio closer in UMAP space compared to Cluster 5. Key original health features driving this assignment include Has_Kidney_Failure, mean_steroid_ng_dl, Has_Kidney_Failure. Cluster membership stability = 1.00.',
 'top_original_features': [{'Original_Feature': 'Has_Kidney_Failure',
   'Correlation': 0.964724305029977,
   'Contribution': 0.321},
  {'Original_Feature': 'mean_steroid_ng_dl',
   'Correlation': 0.4647739244486011,
   'Contribution': 0.246},
  {'Original_Feature': 'Has_Kidney_Failure',
   'Correlation': 0.958453415739

## Test Patient 10

In [67]:
# Example usage for first patient in test set
patient_idx = 10
patient_features = X_test[patient_idx]
stability = stability_test[patient_idx]
explanation = explain_patient_cluster(
    patient_features,
    cent_train,
    umap_feature_names,
    feat_importance_train,
    stability_score=stability
)

import pprint
pprint.pprint(explanation)

{'assigned_cluster': 0,
 'distance_to_assigned': 1.1707468278750008,
 'distance_to_other': 1.3512790845628408,
 'nearest_other_cluster': 5,
 'separation_ratio': 0.15420264432022462,
 'stability_score': 0.94,
 'summary': 'Patient is assigned to Cluster 0 because they are 0.15 separation '
            'ratio closer in UMAP space compared to Cluster 5. Key features '
            'driving this assignment include UMAP_18, UMAP_7, UMAP_19. Cluster '
            'membership stability = 0.94.',
 'top_features_pulling_to_assigned': [{'Contribution': 0.491,
                                       'Feature': 'UMAP_18'},
                                      {'Contribution': 0.396,
                                       'Feature': 'UMAP_7'},
                                      {'Contribution': 0.361,
                                       'Feature': 'UMAP_19'}]}


In [68]:
umap_map = load_umap_feature_map("umap_health_features_with_correlations.csv")

orig_features = translate_umap_to_original(explanation, umap_map)

explanation["top_original_features"] = orig_features

explanation = update_summary(explanation, orig_features)

explanation


{'assigned_cluster': 0,
 'nearest_other_cluster': 5,
 'distance_to_assigned': 1.1707468278750008,
 'distance_to_other': 1.3512790845628408,
 'separation_ratio': 0.15420264432022462,
 'top_features_pulling_to_assigned': [{'Feature': 'UMAP_18',
   'Contribution': 0.491},
  {'Feature': 'UMAP_7', 'Contribution': 0.396},
  {'Feature': 'UMAP_19', 'Contribution': 0.361}],
 'stability_score': 0.94,
 'summary': 'Patient is assigned to Cluster 0 because they are 0.15 separation ratio closer in UMAP space compared to Cluster 5. Key original health features driving this assignment include Has_Kidney_Failure, Has_Kidney_Failure, Has_Kidney_Failure. Cluster membership stability = 0.94.',
 'top_original_features': [{'Original_Feature': 'Has_Kidney_Failure',
   'Correlation': 0.7471517552457481,
   'Contribution': 0.491},
  {'Original_Feature': 'Has_Kidney_Failure',
   'Correlation': 0.9647737840032344,
   'Contribution': 0.396},
  {'Original_Feature': 'Has_Kidney_Failure',
   'Correlation': 0.9179585

## Test Patient 47

In [69]:
# Example usage for first patient in test set
patient_idx = 47
patient_features = X_test[patient_idx]
stability = stability_test[patient_idx]
explanation = explain_patient_cluster(
    patient_features,
    cent_train,
    umap_feature_names,
    feat_importance_train,
    stability_score=stability
)

import pprint
pprint.pprint(explanation)

{'assigned_cluster': 4,
 'distance_to_assigned': 0.869120492545926,
 'distance_to_other': 3.4356442763437847,
 'nearest_other_cluster': 5,
 'separation_ratio': 2.9530126211610854,
 'stability_score': 1.0,
 'summary': 'Patient is assigned to Cluster 4 because they are 2.95 separation '
            'ratio closer in UMAP space compared to Cluster 5. Key features '
            'driving this assignment include UMAP_17, UMAP_8, UMAP_11. Cluster '
            'membership stability = 1.00.',
 'top_features_pulling_to_assigned': [{'Contribution': 0.485,
                                       'Feature': 'UMAP_17'},
                                      {'Contribution': 0.467,
                                       'Feature': 'UMAP_8'},
                                      {'Contribution': 0.237,
                                       'Feature': 'UMAP_11'}]}


In [70]:
umap_map = load_umap_feature_map("umap_health_features_with_correlations.csv")

orig_features = translate_umap_to_original(explanation, umap_map)

explanation["top_original_features"] = orig_features

explanation = update_summary(explanation, orig_features)

explanation


{'assigned_cluster': 4,
 'nearest_other_cluster': 5,
 'distance_to_assigned': 0.869120492545926,
 'distance_to_other': 3.4356442763437847,
 'separation_ratio': 2.9530126211610854,
 'top_features_pulling_to_assigned': [{'Feature': 'UMAP_17',
   'Contribution': 0.485},
  {'Feature': 'UMAP_8', 'Contribution': 0.467},
  {'Feature': 'UMAP_11', 'Contribution': 0.237}],
 'stability_score': 1.0,
 'summary': 'Patient is assigned to Cluster 4 because they are 2.95 separation ratio closer in UMAP space compared to Cluster 5. Key original health features driving this assignment include Has_Kidney_Failure, mean_steroid_ng_dl, blood_macros. Cluster membership stability = 1.00.',
 'top_original_features': [{'Original_Feature': 'Has_Kidney_Failure',
   'Correlation': 0.9412905841920932,
   'Contribution': 0.485},
  {'Original_Feature': 'mean_steroid_ng_dl',
   'Correlation': 0.4647739244486011,
   'Contribution': 0.467},
  {'Original_Feature': 'blood_macros',
   'Correlation': 0.323578178194059,
   'C